In [27]:
from gensim.models import KeyedVectors
import fse # fast sentence embeddings
from fse.models import uSIF
import nltk
import os

from IPython.core.display import display, HTML

In [2]:
path_w2v = '/home/santosh/Work/models/word2vec/CORD-19/'
sif_model = KeyedVectors.load(path_w2v+'CORD-19-p2v_sif_2017.bin')

In [15]:
from spacy import util
import spacy

spacy_model_path = '/home/santosh/Work/models/SpaCy/en-cord19-lg/best/'

print("Loading from", spacy_model_path)
nlp2 = util.load_model_from_path(spacy_model_path)

Loading from /home/santosh/Work/models/SpaCy/en-cord19-lg/best/


In [26]:
col_dict = {}
seven_colours = ['#00FFFF', '#B0C4DE', '#ADFF2F', '#FFA07A', '#E6E6FA', '#FFC0CB', '#FFFF00', '#BC8F8F', '#7FFFD4', '#D8BFD8']
for label, colour in zip(nlp2.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': nlp2.pipe_labels['ner'], 'colors':col_dict}

In [54]:
import requests

def extract_results_from_solr(solr_query_):
    r = requests.get(solr_query_)
    if r.status_code == 200:

        json_data = r.json()
        return json_data

    else:
        print('result_not_suceeded')
        return False
    

def generate_solr_query(query_result):

    list_indexs = []
    similarity_scores = []
    for each_result in query_result:
        list_indexs.append(each_result[0])
        similarity_scores.append(each_result[1])

    solr_query = 'http://localhost:8157/solr/cord19_paragraph_index/select?q=index_key:('
    for each_index in list_indexs:
        solr_query = solr_query+str(each_index)+' '

    solr_query = solr_query+')'

#     print(solr_query)
    
    return solr_query, dict(zip(list_indexs,similarity_scores))

In [88]:
def convert_html(top_results, scores):
    html_markup_list = [None] * len(scores) # [pre define the list, later use to input results based on scores]
    
    for er in top_results['response']['docs']:
        title = '<h2> '+er['title']+'</h2>'
        authors = '<h6> '+er['authors']+'</h6>'
        source = '<a href = "https://doi.org/'+er['source'].strip()+'">[source]</a>'
        paragraph = '<p>'+spacy.displacy.render(nlp2(er['paragraph']), style='ent', jupyter=False, options=options, minify=True)+'<p>'
        
        key_index = list(scores).index(er['index_key'])
        
        document_score = '<i>'+'relevance: '+ str(round(scores[er['index_key']],2))+'</i>'
        
        html_markup_list[key_index] = title+ ' ('+ document_score +')' + '\n'+authors+'\n'+ paragraph+'\n'+source
        
    return html_markup_list

In [89]:
# query = "What is the incubation days of SARS-CoV-2" 

# query = "incubation days coronavirus 2019-nCoV"#  COVID-19
# query = 'socio economic poverty behaviour'

# query = 'what is the influence of pre-existing diseases and comorbidities'
# query = 'public health mitigation measures that could be effective for control'
# query = 'socio-economic and behavioral factors to understand the economic impact of the SARS-CoV-2 virus and whether there were differences. '
# query = 'what are the risk factors for death in COVID-19'
# query = 'what is the basic reproductive number of SARS-CoV-2 in days'
# query = 'what is the serial interval days SARS-CoV-2'
# query = 'what do we know about the environmental factors influencing SARS-CoV-2'
query = 'what do we know about drugs using to treat SARS-CoV-2'
# query = 'Transmission dynamics of the virus SARS-CoV-2'
# query ='risk of fatality among symptomatic hospitalized patients'
# query = 'Efforts targeted at a universal coronavirus vaccine'
# query = ''

query_result = sif_model.sv.similar_by_sentence(nltk.word_tokenize(query), model=sif_model, topn=10)


In [90]:
# generate solr query and get scores
solr_query_url, index_score = generate_solr_query(query_result)
top_results_ = extract_results_from_solr(solr_query_url)


In [91]:
if top_results_:
    display(HTML('<hr>\n'.join(convert_html(top_results_, index_score))))
 
    